In [3]:
import overpy
import requests
import json
import geojson
import geopandas as gpd

overpass_url = "http://overpass-api.de/api/interpreter"
#overpass_query = """
#way["highway"](-12,90,-11,91);
#"""
overpass_query = """
[timeout:10000][maxsize:1073741824][out:json];

( 
    way["highway"]
    (-3.039994,104.664429,-2.862389,104.851540);
);
out geom;
>;
out skel qt;
"""

######## send http request ##########
response = requests.get(overpass_url, 
                        params={'data': overpass_query})

#response 200, means request succeeded.
#print(response) 

response.raise_for_status()

######## json #########
data = response.json()
#response.json() #response.json() only outputs parsed json data, but not in json format

######## json to file #########
import pathlib
pathlib.Path('palembang.json').write_bytes(response.content)

ConnectionError: HTTPConnectionPool(host='overpass-api.de', port=80): Max retries exceeded with url: /api/interpreter?data=%0A%5Btimeout%3A10000%5D%5Bmaxsize%3A1073741824%5D%5Bout%3Ajson%5D%3B%0A%0A%28+%0A++++way%5B%22highway%22%5D%0A++++%28-3.039994%2C104.664429%2C-2.862389%2C104.851540%29%3B%0A%29%3B%0Aout+geom%3B%0A%3E%3B%0Aout+skel+qt%3B%0A (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8f994ed750>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [75]:
######## csv ##########
import csv
import requests

CSV_URL = 'http://samplecsvs.s3.amazonaws.com/Sacramentorealestatetransactions.csv'


with requests.Session() as s:
    download = s.get(CSV_URL)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)

['<?xml version="1.0" encoding="UTF-8"?>']
['<Error><Code>NoSuchBucket</Code><Message>The specified bucket does not exist</Message><BucketName>samplecsvs</BucketName><RequestId>0NYNZ1T7QGESVAEZ</RequestId><HostId>t1EnQ4biTHy8rKNooYJX1Y4ZH0czoxBDyi+Q+/CcOx+wICPzWP2rQnSsuJDVq9TxsXBZ1a2gMvM=</HostId></Error>']


In [ ]:
#$$$$$$$$$$$$$$$ convert json to geojson $$$$$$$$$$$$$$$$$$$
# method 1. system raw convert

#! usr/bin/env python

from sys import argv
from os.path import exists
import simplejson as json 

script, in_file, out_file = argv

data = json.load(open('in_file.json',))

geojson = {
    "type": "FeatureCollection",
    "features": [
    {
        "type": "Feature",
        "geometry" : {
            "type": "Point",
            "coordinates": [[[d["lon"], d["lat"]] for d in elements["geometry"]]]
            },
        "properties" : d,
     } for d in data]
}


output = open('out_file.geojson', 'w')
json.dump(geojson, output)

#print (geojson)

In [11]:
# method 2. dump json as file and convert later 

#with open("SEA_1.geo.json",mode="w") as f:
#    geojson.dump(data,f) 
# this geo.json is NOT geojson file. its a text file that only can be opened with ATOM.
# if save as .geojson, the file cannot be opened by JOSM.
#with open("SEA_2.json",mode="w") as f:
#    json.dump(data,f) 
#error: file importer does not exist

import json
with open('data_m2.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4) #https://stackoverflow.com/questions/12309269/how-do-i-write-json-data-to-a-file
    

#method 3. dump with raw convert first
def convert_json(items):
    import json
    return json.dumps({ "type": "FeatureCollection",
                        "features": [ 
                                        {"type": "Feature",
                                         "geometry": { "type": "Point",
                                                       "coordinates": [ feature['lon'],
                                                                        feature['lat']]},
                                         "properties": { key: value 
                                                         for key, value in feature.items()
                                                         if key not in ('lat', 'lon') }
                                         } 
                                     for feature in json.loads(json.dumps(items))
                                    ]
                       })
data_geojson = convert_json(data)

TypeError: string indices must be integers

In [8]:
# method 4. geopandas

def elem_to_feature(elem):
    return {
        "geometry": {
                "type": "way",
                "coordinates": [[[d["lon"], d["lat"]] for d in elem["geometry"]]]
        },
        "properties": elem["tags"],
    }


# the 'data' variable is from the code in the question
features = [elem_to_feature(elem) for elem in data["elements"]]

gdf = gpd.GeoDataFrame.from_features(features)
gdf.to_file("query_results_2.json", driver="GeoJSON")

KeyError: 'geometry'

In [9]:
# method 5. osmtogeojson

import osmtogeojson
geojson = osmtogeojson('data_m2.json')

TypeError: 'module' object is not callable

In [10]:
from arcgis2geojson import arcgis2geojson
output = arcgis2geojson(data)
# output is empty

In [ ]:
####### next is by overpy method ########

import overpy

api = overpy.Overpass()

# fetch all ways and nodes
result = api.query("""
[timeout:900][maxsize:1073741824][out:json];

( 
    way["highway"]
    (-12,90,-8,94);
);
out geom;
>;
out skel qt;
""")
#print(result)
#geojson = osmtogeojson(result)

# def convert_json(items):
#     import json
#     return json.dumps({ "type": "FeatureCollection",
#                         "features": [ 
#                                         {"type": "Feature",
#                                          "geometry": { "type": "Point",
#                                                        "coordinates": [ feature['lon'],
#                                                                         feature['lat']]},
#                                          "properties": { key: value 
#                                                          for key, value in feature.items()
#                                                          if key not in ('lat', 'lon') }
#                                          } 
#                                      for feature in json.loads(items)
#                                     ]
#                        })
# data_geojson = convert_json(result)